In [135]:
#imports
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

In [136]:
train_inputs = pd.read_csv('../preprocessed_data/train_inputs.csv')
test_inputs = pd.read_csv('../preprocessed_data/test_inputs.csv')
validation_inputs = pd.read_csv('../preprocessed_data/validation_inputs.csv')
train_inputs.head()

,original_Text,Punctuation_Counts_before,Diacritic_Counts,cleaned_text,Stemmed_Text,Lemmatized_Text
0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينو...,146,0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينوف...,وكان رئس وكر ؤقت كسندر تورتشينوف امر سحب قوت ا...,كان رئيس الاوكراني المؤقت الكسندر تورتشينوف مر...
1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,74,1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,حلل عام 2050 حاج مصر الي 21 لير متر كعب حصت حل...,حلول عام 2050 احتاج مصر ال 21 مليار متر مكعب ح...
2,وذكرت وكاله الانباء المحليه جي ان اس ان جماع...,38,0,وذكرت وكاله الانباء المحليه جي ان اس ان جماعه ...,ذكر وكل باء حله جي ان اس ان جمع جيش حمد تشدد ا...,ذكر كال الانباء المحليه جي ان اس ان جماع جيش م...
3,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,378,15,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,وقع خير علي واد عمق 800 متر حاط بثم قمم حده تش...,وقع اختيار علي وادي عمق 800 متر محاط بثماني قم...
4,مسلح حوثي في اب وقال المصدر ان المسلحين الحوث...,26,3,مسلح حوثي اب وقال المصدر ان المسلحين الحوثيين ...,سلح حوث اب وقل صدر ان سلح حوث هجم شطء حرك همي ...,مسلح حوثي اب قال مصدر ان مسلح الحوثيين هاجم نش...


In [137]:
train_labels = pd.read_csv('../preprocessed_data/train_labels.csv')
test_labels = pd.read_csv('../preprocessed_data/test_labels.csv')
validation_labels = pd.read_csv('../preprocessed_data/validation_labels.csv')
train_labels.head()

,original_Text,Punctuation_Counts_before,Diacritic_Counts,cleaned_text
0,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...,3,0,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...
1,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...,8,0,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...
2,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...,3,0,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...
3,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...,9,0,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...
4,أكد مصدر في الحراك التهامي لأبناء محافظه الح...,3,0,أكد مصدر في الحراك التهامي لأبناء محافظه الح...


In [138]:
test_labels.shape[0]

4689

In [139]:
train_size = round(train_labels.shape[0] * 0.2)
test_size = round(test_labels.shape[0] * 0.2)
val_size = round(validation_labels.shape[0] * 0.2)

train_inputs = train_inputs.iloc[:train_size]
train_labels = train_labels.iloc[:train_size]

test_inputs = test_inputs.iloc[:test_size]
test_labels = test_labels.iloc[:test_size]

validation_inputs = validation_inputs.iloc[:val_size]
validation_labels = validation_labels.iloc[:val_size]

In [140]:
train_inputs.shape[0], test_inputs.shape[0], validation_inputs.shape[0]

(7504, 938, 938)

In [141]:
df_train_x = train_inputs.copy().rename(columns={'cleaned_text': 'text'})
df_train_y = train_labels.copy().rename(columns={'cleaned_text': 'summary'})
df_train = pd.concat([df_train_x['text'], df_train_y['summary']], axis=1)

df_test_x = test_inputs.copy().rename(columns={'cleaned_text': 'text'})
df_test_y = test_labels.copy().rename(columns={'cleaned_text': 'summary'})
df_test = pd.concat([df_test_x['text'], df_test_y['summary']], axis=1)

df_val_x = validation_inputs.copy().rename(columns={'cleaned_text': 'text'})
df_val_y = validation_labels.copy().rename(columns={'cleaned_text': 'summary'})
df_validation = pd.concat([df_val_x['text'], df_val_y['summary']], axis=1)
df_train.head()

,text,summary
0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينوف...,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...
1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...
2,وذكرت وكاله الانباء المحليه جي ان اس ان جماعه ...,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...
3,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...
4,مسلح حوثي اب وقال المصدر ان المسلحين الحوثيين ...,أكد مصدر في الحراك التهامي لأبناء محافظه الح...


In [142]:
train_dataset = Dataset.from_dict(df_train)
test_dataset = Dataset.from_dict(df_test)
validation_dataset = Dataset.from_dict(df_validation)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset, "validation": validation_dataset})

In [143]:
from transformers import AutoTokenizer

model = 'moussaKam/AraBART'
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)

In [144]:
max_input_length = 1024
max_target_length = 128

# Tokenizes the input dataframe to match the used AraBART model
# Returns a dictionary {input_ids, attention_mask, summary}
def tokenize(df):
    model_input = [row for row in df['text']]
    model_input = tokenizer(model_input, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(df['summary'], max_length=max_target_length, truncation=True)
    
    model_input['labels'] = labels['input_ids']
    return model_input


### Different lengths??

In [145]:
tokenized_data = dataset.map(tokenize, batched=True)



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


























  0%|          | 0/3 [11:24<?, ?it/s]






In [146]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
pre_trained_model = AutoModelForSeq2SeqLM.from_pretrained(model)

In [147]:
batch_size = 4
model_name = 'AraBART'
arguments = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    num_train_epochs=3,
    predict_with_generate=True
)

In [148]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=pre_trained_model)

In [149]:
trainer = Seq2SeqTrainer(
    pre_trained_model,
    arguments,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [150]:
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 18%|█▊        | 500/2814 [3:51:54<17:50:45, 27.76s/it]

{'loss': 3.7934, 'learning_rate': 4.111584932480455e-05, 'epoch': 0.53}


 33%|███▎      | 938/2814 [7:23:32<14:43:58, 28.27s/it]

{'eval_loss': 2.532254219055176, 'eval_runtime': 627.8905, 'eval_samples_per_second': 1.494, 'eval_steps_per_second': 0.188, 'epoch': 1.0}


 36%|███▌      | 1000/2814 [7:51:53<14:05:08, 27.95s/it] 

...

 89%|████████▉ | 2500/2814 [19:53:14<2:28:50, 28.44s/it]

{'loss': 2.9848, 'learning_rate': 5.579246624022743e-06, 'epoch': 2.67}


100%|██████████| 2814/2814 [22:28:12<00:00, 28.75s/it]

{'eval_loss': 2.48248553276062, 'eval_runtime': 617.5074, 'eval_samples_per_second': 1.519, 'eval_steps_per_second': 0.191, 'epoch': 3.0}
{'train_runtime': 80892.1798, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.035, 'train_loss': 3.2839835464488383, 'epoch': 3.0}


TrainOutput(global_step=2814, training_loss=3.2839835464488383, metrics={'train_runtime': 80892.1798, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.035, 'train_loss': 3.2839835464488383, 'epoch': 3.0})

In [152]:
outputs=trainer.predict(tokenized_data['test'])

100%|██████████| 118/118 [25:41<00:00, 13.06s/it]


In [153]:
out_str=tokenizer.batch_decode(outputs.predictions, skip_special_tokens=True)

In [154]:
#!pip install rouge

In [159]:
out_str

['ادين الرائد نضال حسن البالغ من العمر 42 عاما بقتل 13 جنديا امريكيا وجرح اخرين',
 'قال وزير الاقتصاد الألماني إن بلاده قد تقدم خطوه جديده للحد من تدفق طالبي اللجوء إلي',
 'قال المرصد السوري لحقوق الانسان المعارض إن قوات الجيش السوري استعادت حي الخالديه الواقع جنوب مدينه',
 'فاز نادي أرسنال الإنجليزي علي ضيفه هال سيتي بهدفين مقابل لاشيء في المباراه التي',
 'يعيش المسيحيون في العراق في مناطق متنازع عليها بين الحكومه المركزيه وإقليم',
 'سيطرت القوات الحكوميه السوريه علي بلده القريتين الواقعه في ريف حمص الشرقي بعد',
 'تنظر محكمه إسرائيليه في قضيه الجدار الفاصل بين قريه بتير الفلسطينيه',
 'أصدرت محكمه عسكريه في قطاع غزه حكما بالسجن المؤبد علي محمود السلفيتي و',
 'قالت قوه حفظ السلام التابعه للأمم المتحده في دارفور إنها تعرضت لاطلاق نار من',
 'تناولت الصحف البريطانيه الصادره صباح اليوم الجمعه عددا من القضايا السياسيه والاجتماعيه',
 'قال رئيس الوزراء الإسرائيلي بنيامين نتنياهو إن بلاده لن تسمح لإيران بتطوير أسلحه نوويه',
 'قال محامي الاتحاد الدولي لكره القدم فيفا مايكل لوبر إن التحقيقات التي

In [174]:
from joblib import dump, load
dump(trainer.model, 'model.joblib')
dump(outputs, 'outputs_tokenized.joblib')
dump(out_str, 'outputs_text.joblib')
# to load the model again

# trainer.model = load('model.joblib')

['outputs_text.joblib']

In [160]:
from rouge import Rouge


score = Rouge().get_scores(out_str, tokenized_data['test']['summary'])

for i in range(0, 3):
    print("rouge-1 :" , score[i]['rouge-1']['f']*100)
    print("rouge-2 :" , score[i]['rouge-2']['f']*100)
    print("rouge-l :" , score[i]['rouge-l']['f']*100)
    print("")

rouge-1 : 20.83333290364584
rouge-2 : 8.163264897959204
rouge-l : 16.66666623697918

rouge-1 : 15.789473196675916
rouge-2 : 5.405404923301723
rouge-l : 10.52631530193908

rouge-1 : 38.88888840277778
rouge-2 : 11.42857093877553
rouge-l : 38.88888840277778

